In [1]:
!pip install pykrx

실습: 삼성전자 주가 데이터를 분석하는 어플리케이션을 function으로 수정하여, 다른 종목의 데이터를 정해진 내용대로 가져와서 답하는 어플리케이션을 작성하시오.

In [14]:
import os
import openai
from pykrx import stock
import json

openai.api_key = os.getenv("OPENAI_API_KEY")

def get_analysis_stock(ticker, start_date, end_date):
    try:
        # 주가 데이터 가져오기
        df = stock.get_market_ohlcv(start_date, end_date, ticker)

        # print(df.to_string())
        return json.dumps(df.to_string())

    except Exception as e:
        return f"분석 중 오류가 발생했습니다: {e}"
    
def analyze_samsung_stock(start_date, end_date):
    """
    삼성전자 주가 데이터를 가져와서 분석 정보를 제공합니다.

    Args:
        start_date (str): 시작 날짜 (YYYYMMDD 형식)
        end_date (str): 종료 날짜 (YYYYMMDD 형식)

    Returns:
        str: 분석 결과 문자열
    """

    try:
        ticker = "005930"
        
        df_string = get_analysis_stock(ticker, start_date, end_date)
        
        prompt = f"""
                다음은 {ticker} 주식의 {start_date}부터 {end_date}까지의 주가 데이터입니다.
        {df_string}
        이 데이터를 분석하고 주요 트렌드와 인사이트를 제공하십시오.
        """
        
        messages = [{"role": "system", "content": "주식 분석 전문가"}, {"role": "user", "content": prompt}]
        functions = [
            {
                "name": "get_analysis_stock",
                "description": "주식 데이터를 분석하여 주요 트렌드와 인사이트를 제공합니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string", "description": "주가 데이터를 가져올 종목의 티커"},
                        "start_date": {"type": "string", "description": "시작 날짜 (YYYYMMDD 형식)"},
                        "end_date": {"type": "string", "description": "끝 날짜 (YYYYMMDD 형식)"},
                    },
                    "required": ["ticker", "start_date", "end_date"],
                },
            }
        ]
        
        response = openai.chat.completions.create(  # ChatCompletion 사용
            model="gpt-4o-mini",  # 또는 다른 적절한 모델
            messages=messages,
            functions=functions,
            function_call="auto"
        )
        
        analysis = response.choices[0].message

        if analysis.function_call is not None:
            # Note: the JSON response may not always be valid; be sure to handle errors
            available_functions = {
                "get_analysis_stock": get_analysis_stock,
            }
            function_name = analysis.function_call.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(analysis.function_call.arguments)
            function_response = function_to_call(
                ticker=function_args['ticker'],
                start_date=function_args['start_date'],
                end_date=function_args['end_date']
            )

            messages.append(analysis)
            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                }
            )
            second_response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
            )  # get a new response from GPT where it can see the function response

            print(second_response.choices[0].message.content)

    except Exception as e:
        return f"분석 중 오류가 발생했습니다: {e}"


# 예시 사용
start_date = "20230101"
end_date = "20231231"

# ChatGPT 연동 예시 (가상)
user_query = "2023년 삼성전자 주가 분석해줘"
start_date, end_date = "20230101", "20231231" # 날짜 추출 로직 필요

analyze_samsung_stock(start_date, end_date)


### 005930 (삼성전자) 주식 분석 (2023)

#### 1. 주가 변동 및 주요 트렌드
- **시작가**: 2023년 1월 2일 기준 55,500원
- **종료가**: 2023년 12월 28일 기준 78,500원
- **최고가**: 2023년 5월 30일 72,300원
- **최저가**: 2023년 1월 2일 55,500원

#### 2. 월별 주가 동향
- **1월**: 주가는 55,500원에서 시작해 61,800원으로 상승. 19일에 큰 상승(1.82%)을 보였고, 월말에 63,300원에 마감.
- **2월**: 63,500원에서 시작하여 전반적으로 하락세를 기록. 월말에는 60,600원에 마감.
- **3월**: 61,500원에서 시작, 중반에 61,300원으로 하락 후 반등하며 64,000원으로 마감.
- **4월**: 63,900원에서 시작해 66,100원으로 상승하며 상승세 유지.
- **5월**: 65,900원 시작하여 72,300원까지 증가함. 19일엔 최고가를 기록(3.32% 상승).
- **6월**: 주가는 약세로 전환, 71,700원 시작하여 70,000원대에서 머물며 하락세를 이어감.
- **7월**: 73,000원에서 시작하여 불안정한 움직임 보였고 69,800원으로 마감.
- **8월**: 지속적인 하락세, 67,300원에서 시작하여 68,800원까지 하락.
- **9월**: 68,900원에서 시작하여 하락과 상승을 반복, 69,400원으로 종료.
- **10월**: 70,000원에서 시작해 하락세 계속, 66,000원으로 마감.
- **11월**: 반등세 시작, 68,600원에서 상승하며 73,300원으로 마감.
- **12월**: 73,000원에서 시작하여 점진적으로 상승하며 78,500원으로 마감.

#### 3. 주가 변동폭
- 가장 큰 상승폭은 5월 30일과 19일에 발생했으며, 특히 5월 19일에는 3.32% 상승.
- 9월 18일부터 20일 사이에는 2.5% 하락을 기록, 연내에 여러 차례 하락과 상승을 반복함.